In [ ]:
from dotenv import load_dotenv
import os

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools import Tool

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model


In [ ]:
llm = init_chat_model(
    model="amazon/nova-2-lite-v1:free",
    model_provider="openai",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0.6
)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
texts = [
    "ServiceDesk Plus allows administrators to automate workflows, manage incidents, and handle IT tickets efficiently.",
    "In Acronis Cyber Protect, backup failures may occur due to agent misconfiguration, network interruption, or quota exhaustion.",
    "LangGraph is a state machine for AI agents that ensures predictable, reproducible workflows with memory and branching logic.",
    "FAISS uses approximate nearest neighbor search to retrieve similar embeddings in large-scale vector databases.",
    "Office 365 onboarding includes creating a new user, assigning licenses, and configuring access permissions through Entra ID.",
]


In [ ]:
vector_store=FAISS.from_texts(texts, embedding=embeddings)

In [ ]:
print(vector_store.similarity_search('servicedesk',k=3))

In [ ]:
retriever=vector_store.as_retriever(search_kwargs={'k': 3})

In [ ]:
def kb_search(query: str):
    docs = retriever.invoke(query)  
    return "\n\n".join(d.page_content for d in docs)

retriever_tool = Tool(
    name="kb_search",
    description="Search the knowledge base for software-related information",
    func=kb_search
)

In [ ]:
agent = create_agent(
    model=llm,
    tools=[retriever_tool],
    system_prompt=
        """You are a helpful assistant. Answer user queries using the kb_search tool. 
        You may need to use it multiple times before answering."""
    
)



In [ ]:
response = agent.invoke(
    {
        "messages": [
            ("user", "What do you think about ServiceDesk Plus and Acronis?")
        ]
    }
) 
print(response["messages"][-1].content)